Conclusion-LDA is not helpful in this situation

Things to try to increase performance: 
-even more data
-separate models for different genres
-topic model
-ridge regression, lasso regression
-bi grams, n grams
-sentiment analysis
-Look for transformations opps, etc
-model only using genre

In [107]:
import pandas as pd
from textblob import TextBlob
from textblob import Word
from gensim import corpora, models, similarities
import os
import tempfile
import gensim
from sklearn.linear_model import LinearRegression
import math
import numpy as np
import xgboost as xgb
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from bayes_opt import BayesianOptimization
from sklearn.metrics import mean_squared_error
from gensim import corpora, models, similarities
from gensim.models.ldamulticore import LdaMulticore
import gensim

TEMP_FOLDER = tempfile.gettempdir()

In [108]:
links = pd.read_csv('/Users/scottgleave/Downloads/DataMiningCourse/finalprojectdata/links2.csv', header = None)
text = pd.read_csv('/Users/scottgleave/Downloads/DataMiningCourse/finalprojectdata/reviewtext.csv', header = None)

In [109]:
links2 = pd.read_csv('/Users/scottgleave/Downloads/DataMiningCourse/finalprojectdata/links3.csv', header = None)
text2 = pd.read_csv('/Users/scottgleave/Downloads/DataMiningCourse/finalprojectdata/reviewtext2.csv', header = None)

In [110]:
#prep 2nd set of data, which has lots of links that didn't work

links2[5] = text2
links2.drop_duplicates([5], inplace = True)
links2 = links2[links2[5] != '[]']
links2 = links2[links2[5] != '[]']
links2.dropna(subset = [5],inplace = True)

In [111]:
links[5] = text

In [112]:
links.drop_duplicates([5], inplace = True)


In [113]:
links = pd.concat([links,links2], axis = 0)
links = links.reset_index()
links = links.drop('index', axis = 1)

In [114]:
links.shape

(2903, 6)

In [115]:
text = links[5]
text.shape

(2903,)

In [116]:
#lowercase text
text = text.apply(lambda x: " ".join(x.lower() for x in x.split()))
text.head()

0    ["with all of the development muscle behind ea...
1    ['when i first reviewed civilization vi back i...
2    [' ', 'there aren’t many games that make me fe...
3    ['overkill’s ', ' is an earnest attempt to del...
4    ["it's often with rose-colored glasses that i ...
Name: 5, dtype: object

In [117]:
#remove brackets
text = text.apply(lambda x: x[1:-1])
text.head()

0    "with all of the development muscle behind ea ...
1    'when i first reviewed civilization vi back in...
2    ' ', 'there aren’t many games that make me fee...
3    'overkill’s ', ' is an earnest attempt to deli...
4    "it's often with rose-colored glasses that i l...
Name: 5, dtype: object

In [118]:
#remove new line
text = text.replace(r'\\n','', regex=True)
text.head()

0    "with all of the development muscle behind ea ...
1    'when i first reviewed civilization vi back in...
2    ' ', 'there aren’t many games that make me fee...
3    'overkill’s ', ' is an earnest attempt to deli...
4    "it's often with rose-colored glasses that i l...
Name: 5, dtype: object

In [119]:
#remove punc
text = text.str.replace('[^\w\s]','')
text.head()

0    with all of the development muscle behind ea d...
1    when i first reviewed civilization vi back in ...
2      there arent many games that make me feel as ...
3    overkills   is an earnest attempt to deliver a...
4    its often with rosecolored glasses that i look...
Name: 5, dtype: object

In [120]:
#remove stop words
from nltk.corpus import stopwords
stop = stopwords.words('english')
text = text.apply(lambda x: " ".join(x for x in x.split() if x not in stop))
text.head()

0    development muscle behind ea dices massive sho...
1    first reviewed civilization vi back 2016 award...
2    arent many games make feel cool warframe im bu...
3    overkills earnest attempt deliver cooperative ...
4    often rosecolored glasses look back pokemons k...
Name: 5, dtype: object

In [121]:
freq = pd.Series(' '.join(text[0]).split()).value_counts()[-5:]
#remove uncommon words
freq = list(freq.index)
text = text.apply(lambda x: " ".join(x for x in x.split() if x not in freq))
text.head()

0    development muscle behind ea dices massive sho...
1    first reviewed civilization vi back 2016 award...
2    arent many games make feel cool warframe im bu...
3    overkills earnest attempt deliver cooperative ...
4    often rosecolored glasses look back pokemons k...
Name: 5, dtype: object

In [122]:
###remove common words###
freq = pd.Series(' '.join(text[0]).split()).value_counts()[:10]
freq = list(freq.index)
text = text.apply(lambda x: " ".join(x for x in x.split() if x not in freq))
#Visualize
text.head()

0    development muscle behind ea dices massive sho...
1    first reviewed civilization vi back 2016 award...
2    arent many games make feel cool warframe im bu...
3    overkills earnest attempt deliver cooperative ...
4    often rosecolored glasses look back pokemons k...
Name: 5, dtype: object

In [123]:
###Spelling correction###

text[:5].apply(lambda x: str(TextBlob(x).correct()))

###Tokenization (dividing the text into a sequence of words or sentences)###
TextBlob(text[0][1]).words

#Stemming (removal of suffices, like “ing”, “ly”, “s”, etc. by a simple rule-based approach)
#from nltk.stem import PorterStemmer
#st = PorterStemmer()
#train['text'][:5].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))

#Lemmatization (more effective option than stemming because 
# converts the word into its root word, rather than just stripping the suffices)
#*Preferred to stemming*#


text = text.apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
text.head()

0    development muscle behind ea dice massive shoo...
1    first reviewed civilization vi back 2016 award...
2    arent many game make feel cool warframe im bul...
3    overkill earnest attempt deliver cooperative a...
4    often rosecolored glass look back pokemons kan...
Name: 5, dtype: object

In [124]:
text = text.to_frame()
text['score'] = links[1]

In [125]:
text.head()

,5,score
0,development muscle behind ea dice massive shoo...,7.5
1,first reviewed civilization vi back 2016 award...,9.4
2,arent many game make feel cool warframe im bul...,8.6
3,overkill earnest attempt deliver cooperative a...,5.3
4,often rosecolored glass look back pokemons kan...,8.3


In [126]:
#train val split

trainval = text.sample(800)
train = text.loc[~text.index.isin(trainval.index), :]
y = train['score']
valy = trainval['score']
trainval.drop(['score'], axis = 1, inplace = True)
train.drop(['score'], axis = 1, inplace = True)

/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [127]:
#for train values
textlist = train.values.tolist()

In [128]:
textlist = [str(word) for word in textlist]

In [129]:
texts = [[word for word in document.lower().split() ]
         for document in textlist]

In [130]:
dictionary = corpora.Dictionary(texts)

In [131]:
dictionary.save(os.path.join(TEMP_FOLDER, 'igntfidf.dict'))

In [132]:
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize(os.path.join(TEMP_FOLDER, 'igntfidf.mm'), corpus)  # store to disk, for later use

In [177]:
avals = [.01,.01,.01,.01,.01,.01,.01,.01,.01,.01]
lda = LdaMulticore(corpus, id2word=dictionary, num_topics=10, alpha = avals)

In [178]:
trainlda = lda[corpus]
topicstrain = lda.get_document_topics(corpus)
all_topics_csrtrain = gensim.matutils.corpus2csc(topicstrain)
all_topics_numpytrain = all_topics_csrtrain.T.toarray()

In [179]:
#validation text
new_doc = trainval[5]
valtext = new_doc.values.tolist()
valtext = [str(word) for word in valtext]
valtext = [[word for word in document.lower().split() ]
         for document in valtext]
valcorpus = [dictionary.doc2bow(text) for text in valtext]

In [180]:
#generate topics on validation
trainlda = lda[valcorpus]
topicsval = lda.get_document_topics(valcorpus)
all_topics_csrval = gensim.matutils.corpus2csc(topicsval)
all_topics_numpyval = all_topics_csrval.T.toarray()

In [165]:
all_topics_numpyval.shape

(800, 10)

In [139]:
all_topics_numpyval[0]

array([0.        , 0.07302336, 0.        , 0.05029746, 0.2143739 ,
       0.58332884, 0.        , 0.02358942, 0.01867567, 0.03604301])

In [181]:
all_topics_numpyval[0]

array([0.29204679, 0.        , 0.        , 0.        , 0.        ,
       0.4166424 , 0.        , 0.        , 0.29115459, 0.        ])

## XGBoost

In [192]:
#bayesian opt
yarray = y.values
dtrain = xgb.DMatrix(all_topics_numpy, label=yarray)
def xgb_evaluate(max_depth, gamma, colsample_bytree):
    params = {'eval_metric': 'rmse',
              'max_depth': int(max_depth),
              'subsample': 0.8,
              'eta': 0.1,
              'gamma': gamma,
              'colsample_bytree': colsample_bytree}
    # Used around 1000 boosting rounds in the full model
    cv_result = xgb.cv(params, dtrain, num_boost_round=100, nfold=3)    
    
    # Bayesian optimization only knows how to maximize, not minimize, so return the negative RMSE
    return -1.0 * cv_result['test-rmse-mean'].iloc[-1]

xgb_bo = BayesianOptimization(xgb_evaluate, {'max_depth': (3, 7),
                                            'gamma': (0, 1),
                                            'colsample_bytree': (0.3, 0.9)})

xgb_bo.maximize(init_points=3, n_iter=5, acq='ei')



NameError: name 'all_topics_numpy' is not defined

In [357]:
param = xgb_bo.max
param

{'target': -1.1006746666666667,
 'params': {'colsample_bytree': 0.8976729774092942,
  'gamma': 0.9971962663965198,
  'max_depth': 4.321612175930944}}

In [193]:
yarray = y.values
xgbreg = xgb.XGBRegressor()
xgbreg.fit(all_topics_numpytrain,yarray)

predictions2 = xgbreg.predict(all_topics_numpyval)
predictionsdf2 = pd.DataFrame(predictions2)
valy = valy.reset_index()
valy.drop(['index'], axis = 1, inplace = True)
predictionsdf2['true'] = valy
predictionsdf2['diff'] = predictionsdf2['true'] - predictionsdf2[0]
predictionsdf2['diff'] = predictionsdf2['true'] - predictionsdf2[0]
predictionsdf2['diff'] = predictionsdf2['diff']**2
diff = predictionsdf2['diff'].values
diff = np.mean(diff)
diff = math.sqrt(diff)
diff

1.5305064131143693

## Lasso Regression

In [338]:
#find best alpha for lasso regression

alphas = [1e-15, 1e-10, 1e-8, 1e-5,1e-4, 1e-3,1e-2, 1, 5, 10]
alphas2 = [1e-5,1.5e-5,1e-4, 1.5e-4,1e-3]
def runlasso(alpha):
    global valy
    yarray = y.values
    lasso = Lasso(alpha = alpha, max_iter=1e5)
    lasso.fit(all_topics_numpy,yarray)

    predictions3 = lasso.predict(all_topics_numpytest)
    predictionsdf3 = pd.DataFrame(predictions3)
    valy = valy.reset_index()
    valy.drop(['index'], axis = 1, inplace = True)
    predictionsdf3['true'] = valy
    predictionsdf3['diff'] = predictionsdf3['true'] - predictionsdf3[0]
    predictionsdf3['diff'] = predictionsdf3['true'] - predictionsdf3[0]
    predictionsdf3['diff'] = predictionsdf3['diff']**2
    diff = predictionsdf3['diff'].values
    diff = np.mean(diff)
    diff = math.sqrt(diff)
    return diff
    
test = [runlasso(x) for x in alphas2]
print(test)
    
    
    

[1.1252147868577373, 1.1126873118616674, 1.0177467770085127, 0.9907478349117224, 1.1576718275231785]


In [339]:
yarray = y.values
lasso = Lasso(alpha = 1.5e-4, max_iter=1e5)
lasso.fit(all_topics_numpy,yarray)

predictions3 = lasso.predict(all_topics_numpytest)
predictionsdf3 = pd.DataFrame(predictions3)
valy = valy.reset_index()
valy.drop(['index'], axis = 1, inplace = True)
predictionsdf3['true'] = valy
predictionsdf3['diff'] = predictionsdf3['true'] - predictionsdf3[0]
predictionsdf3['diff'] = predictionsdf3['true'] - predictionsdf3[0]
predictionsdf3['diff'] = predictionsdf3['diff']**2
diff = predictionsdf3['diff'].values
diff = np.mean(diff)
diff = math.sqrt(diff)
diff


0.9907478349117224

## OLS

In [182]:
yarray = y.values
lm = LinearRegression()
lm.fit(all_topics_numpytrain,yarray)
predictions = lm.predict(all_topics_numpyval)
predictionsdf = pd.DataFrame(predictions)

In [183]:
predictionsdf.describe()

,0
count,800.000000
mean,7.341797
std,0.094536
min,6.994296
25%,7.282793
50%,7.341989
75%,7.400021
max,7.716115


In [184]:
valy = valy.reset_index()
valy

,index,score
0,0,8.0
1,1,5.2
2,2,7.0
3,3,9.0
4,4,9.6
5,5,8.4
6,6,6.0
7,7,8.3
8,8,8.3
9,9,8.5


In [185]:
valy.drop(['index'], axis = 1, inplace = True)
valy

,score
0,8.0
1,5.2
2,7.0
3,9.0
4,9.6
5,8.4
6,6.0
7,8.3
8,8.3
9,8.5


In [186]:
predictionsdf['true'] = valy
predictionsdf

,0,true
0,7.319201,8.0
1,7.276212,5.2
2,7.231765,7.0
3,7.450518,9.0
4,7.333216,9.6
5,7.513257,8.4
6,7.209435,6.0
7,7.429515,8.3
8,7.409582,8.3
9,7.381310,8.5


In [187]:
predictionsdf['diff'] = predictionsdf['true'] - predictionsdf[0]
predictionsdf

,0,true,diff
0,7.319201,8.0,0.680799
1,7.276212,5.2,-2.076212
2,7.231765,7.0,-0.231765
3,7.450518,9.0,1.549482
4,7.333216,9.6,2.266784
5,7.513257,8.4,0.886743
6,7.209435,6.0,-1.209435
7,7.429515,8.3,0.870485
8,7.409582,8.3,0.890418
9,7.381310,8.5,1.118690


In [188]:
#let's also plot histogram of scores




In [189]:
#residual plot here




In [190]:
#RMSE
predictionsdf['diff'] = predictionsdf['true'] - predictionsdf[0]
predictionsdf['diff'] = predictionsdf['diff']**2
diff = predictionsdf['diff'].values
diff = np.mean(diff)
diff = math.sqrt(diff)
diff

#diff = predictionsdf['diff'].values.tolist()
#diff = math.sqrt(np.mean(diff**2))
#diff

1.5099916235548214

In [191]:
arravg = np.mean(yarray)
predictionsdf['avg'] = arravg
predictionsdf['avgdiff'] = predictionsdf['avg'] - predictionsdf['true']
predictionsdf['avgdiff'] = predictionsdf['avgdiff']**2
avgdiff = predictionsdf['avgdiff'].values
avgdiff = np.mean(avgdiff)
avgdiff = math.sqrt(avgdiff)
avgdiff

1.5204950925527425